In [1]:
import h5py
import torch
from src.data.play import Play, slice_keys
from src.utilities.embedding_utilities import n_players
from torch.nn import Embedding
from torch import Tensor as t

In [2]:
db = h5py.File("cache/full_db.h5", "r")

In [3]:
i = 0
plays = []
for p in db["2016_regular/plays"][0:10]:
    play = Play(p)
    plays.append(play)

In [4]:
play

jeff-teague missed 2 point LongMidRange | Defensive rebound by andre-drummond 

In [5]:
print(play.triggers)

{'shooter': [{'initial_event': 'shot'}], 'shot_made': [{'initial_event': 'shot'}], 'is_3': [{'initial_event': 'shot'}], 'shot_distance': [{'initial_event': 'shot'}], 'shot_fouled': [{'initial_event': 'shot'}], 'shooting_fouler': [{'shot_fouled': True}], 'assisted': [{'initial_event': 'shot'}], 'assister': [{'initial_event': 'shot'}], 'blocked': [{'shot_made': False}], 'blocker': [{'blocked': True}], 'turnoverer': [{'initial_event': 'turnover'}], 'offensive_foul': [{'initial_event': 'turnover'}], 'offensive_fouler': [{'offensive_foul': True}], 'offensive_foul_drawer': [{'offensive_foul': True}], 'stolen': [{'initial_event': 'turnover'}], 'stealer': [{'stolen': True}], 'over_limit_fouler': [{'initial_event': 'foul_over_limit'}], 'over_limit_foul_drawer': [{'initial_event': 'foul_over_limit'}], 'first_free_throw_made': [{'initial_event': 'foul_over_limit'}, {'shot_fouled': True, 'shot_made': False}], 'middle_free_throw_made': [{'shot_fouled': True, '3_point_shot': True, 'shot_made': False

In [7]:
from collections import defaultdict
items_per_type = defaultdict(list)
for k, v in play.types.items():
    items_per_type[v].append(k)

print(items_per_type)

defaultdict(<class 'list'>, {'embedding': ['offense_team', 'defense_team', 'shooter', 'shooting_fouler', 'assister', 'blocker', 'turnoverer', 'offensive_fouler', 'offensive_foul_drawer', 'stealer', 'common_fouler', 'common_foul_drawer', 'over_limit_fouler', 'over_limit_foul_drawer', 'free_thrower', 'offensive_rebounder', 'defensive_rebounder'], 'embedding_list': ['offense_roster', 'defense_roster'], 'bool': ['is_second_chance', 'shot_made', 'is_3', 'shot_fouled', 'assisted', 'blocked', 'offensive_foul', 'stolen', 'first_free_throw_made', 'middle_free_throw_made', 'last_free_throw_made'], 'choice': ['initial_event', 'shot_type', 'rebound_type'], 'scalar': ['shot_distance']})
